<a href="https://colab.research.google.com/github/0jipy/192kbps_colab_JIPY/blob/main/__02_%ED%85%8D%EC%8A%A4%ED%8A%B8%EC%83%9D%EC%84%B1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# RNN을 이용한 텍스트 생성
- '경마장에 있는 말이 뛰고 있다'
- '그의 말이 법이다'
- '가는 말이 고와야 오는 말이 곱다'

In [1]:
import numpy as np 
import tensorflow as tf
seed = 2022
np.random.seed(seed)
tf.random.set_seed(seed)

In [2]:
text = '''경마장에 있는 말이 뛰고 있다\n
그의 말이 법이다\n
가는 말이 고와야 오는 말이 곱다\n'''

## 텍스트 전처리

In [3]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

In [5]:
# 단어 집합 생성
t = Tokenizer()
t.fit_on_texts([text])

In [6]:
t.word_index

{'가는': 8,
 '경마장에': 2,
 '고와야': 9,
 '곱다': 11,
 '그의': 6,
 '뛰고': 4,
 '말이': 1,
 '법이다': 7,
 '오는': 10,
 '있는': 3,
 '있다': 5}

In [7]:
# 단어 집합 크기 - Keras Tokenizer의 시작 인덱스가 1이기 때문
vocab_size = len(t.word_index) + 1
vocab_size

12

In [9]:
t.texts_to_sequences(['그의 말이 법이다'])

[[6, 1, 7]]

In [12]:
# 학습에 사용될 샘플 시퀀스
sequences = []
for line in text.split('\n'):
    encoded = t.texts_to_sequences([line])[0]
    for i in range(1, len(encoded)):
        sequence = encoded[:i+1]
        sequences.append(sequence)
sequences

[[2, 3],
 [2, 3, 1],
 [2, 3, 1, 4],
 [2, 3, 1, 4, 5],
 [6, 1],
 [6, 1, 7],
 [8, 1],
 [8, 1, 9],
 [8, 1, 9, 10],
 [8, 1, 9, 10, 1],
 [8, 1, 9, 10, 1, 11]]

In [13]:
max_len = max(len(s) for s in sequences)
max_len

6

## Padding
- 전체 샘플의 길이를 가장 긴 샘플의 길이(6)로 패딩
- 'pre' 옵션을 주면 앞을 0으로 채움

In [14]:
sequences = pad_sequences(sequences, maxlen=max_len, padding='pre')
sequences[:5]

array([[0, 0, 0, 0, 2, 3],
       [0, 0, 0, 2, 3, 1],
       [0, 0, 2, 3, 1, 4],
       [0, 2, 3, 1, 4, 5],
       [0, 0, 0, 0, 6, 1]], dtype=int32)

In [15]:
X = sequences[:,:-1] # 타겟설정의 이유를 잘 모르겠네
y = sequences[:,-1]

In [16]:
Y = to_categorical(y)       # to_categorical(y, vocab_size)
Y[:3]

array([[0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.]], dtype=float32)

## 모델 정의

- Embedding
- SimpleRNN

In [17]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, SimpleRNN, Dense

In [20]:
# Embedding=4, SimpleRNN=32
model = Sequential([
    Embedding(vocab_size, 4, input_length=max_len-1),
    SimpleRNN(32),
    Dense(vocab_size, activation='softmax')
])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 5, 4)              48        
                                                                 
 simple_rnn_1 (SimpleRNN)    (None, 32)                1184      
                                                                 
 dense (Dense)               (None, 12)                396       
                                                                 
Total params: 1,628
Trainable params: 1,628
Non-trainable params: 0
_________________________________________________________________


## 모델 설정/학습

In [21]:
model.compile(
    optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy']
)

In [22]:
hist = model.fit(X, Y, epochs=200, verbose=0)
hist.history['accuracy'][-1] # 어큐러시 [-1]??
# 샘자료 0.90하고는 대비되는데, 뭐가 잘못됐나?

1.0

## 모델 검증

In [25]:
from my_util import sentence_generation

In [26]:
# 모델, 토크나이저, 현재 단어, 반복할 횟수
sentence_generation(model, t, '경마장에', 3)

TypeError: ignored

In [28]:
sentence_generation(model, t, '그의', 2)

TypeError: ignored

In [31]:
sentence_generation(model, t, '가는', 5)

TypeError: ignored

## 모델 변화
- Embedding vector 갯수: 2, 4, 6
- RNN 노드 갯수: 24, 32, 48

In [32]:
def execute_model(n_embed, n_rnn):
    model = Sequential([
        Embedding(vocab_size, n_embed, input_length=max_len-1),
        SimpleRNN(n_rnn),
        Dense(vocab_size, activation='softmax')
    ])
    model.compile(
        optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy']
    )
    history = model.fit(X, Y, epochs=200, verbose=0)
    print(f"정확도: {history.history['accuracy'][-1]:.4f}")
    print(sentence_generation(model, t, max_len, '경마장에', 3))
    print(sentence_generation(model, t, max_len, '그의', 2))
    print(sentence_generation(model, t, max_len, '가는', 5))

In [33]:
for n_embed in [2, 4, 6]:
    for n_rnn in [24, 36, 48]:
        print('======================================')
        print(f'n_embed={n_embed}, n_rnn={n_rnn}')
        execute_model(n_embed, n_rnn)
        print()

n_embed=2, n_rnn=24
정확도: 0.8182
경마장에 말이 말이 오는
그의 말이 법이다
가는 말이 법이다 오는 말이 곱다

n_embed=2, n_rnn=36
정확도: 0.8182
경마장에 말이 말이 오는
그의 말이 법이다
가는 말이 법이다 오는 말이 곱다

n_embed=2, n_rnn=48
정확도: 0.9091
경마장에 말이 고와야 오는
그의 말이 법이다
가는 말이 고와야 오는 말이 곱다

n_embed=4, n_rnn=24
정확도: 0.8182
경마장에 말이 말이 오는
그의 말이 법이다
가는 말이 법이다 오는 말이 곱다

n_embed=4, n_rnn=36
정확도: 1.0000
경마장에 있는 말이 뛰고
그의 말이 법이다
가는 말이 고와야 오는 말이 곱다

n_embed=4, n_rnn=48
정확도: 0.9091
경마장에 말이 고와야 오는
그의 말이 법이다
가는 말이 고와야 오는 말이 곱다

n_embed=6, n_rnn=24
정확도: 1.0000
경마장에 있는 말이 뛰고
그의 말이 법이다
가는 말이 고와야 오는 말이 곱다

n_embed=6, n_rnn=36
정확도: 1.0000
경마장에 있는 말이 뛰고
그의 말이 법이다
가는 말이 고와야 오는 말이 곱다

n_embed=6, n_rnn=48
정확도: 1.0000
경마장에 있는 말이 뛰고
그의 말이 법이다
가는 말이 고와야 오는 말이 곱다

